# Labs Access Analysis: 01 Data Download

## Setup
import all the needed libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from openrouteservice import client
import time
import json
import os

In [16]:
# create the directory with the data
if not os.path.exists('data'):
    os.makedirs('data')

In [36]:
url_data_labs = "https://openp2pdesign.github.io/labs_access_analysis/data/labs.csv"

##REMOVE this line when you are ready to publish the data
url_data_labs = "data/labs.csv"
api_key_ors = '' # write here your API key from OpenRouteService
                 # you can get from here https://openrouteservice.org/

## data cleaning

In [19]:
labs = pd.read_csv(url_data_labs)

show the first 5 lines

In [3]:
labs.head(5)

,Name,Type,City,Cluster,Address,CAP,Lat,Long,Source,URL,Notes
0,Fab Lab Barcelona,Fab Lab,Barcelona,BCN,"Carrer de Pujades, 102",8005.0,41.396925,2.194335,Fablabs.io,https://fablabbcn.org/,NaN
1,MADE Makerspace Barcelona,Fab Lab,Barcelona,BCN,Carrer Noguera Pallaresa 59-61,8014.0,41.373238,2.141084,"Fablabs.io, Hackerspaces.org, Makerspaces.make.co",NaN,NaN
2,Soko Tech,Fab Lab,Barcelona,BCN,"Carrer de Vallès i Ribot, 36",8027.0,41.424535,2.188883,Fablabs.io,https://soko.tech/,NaN
3,LSCongres,Fab Lab,Barcelona,BCN,"Carrer del Cardenal Tedeschini, 50",8027.0,41.426002,2.182026,Fablabs.io,https://fablabs.io/labs/lscongres,NaN
4,Green Fab Lab,Fab Lab,Cerdanyola del Vallès,BCN,"Ctra. BV-1415 (Horta-Cerdanyola), km 7",8290.0,41.450394,2.133441,Fablabs.io,NaN,NaN


### description of the fields of the dataset
|  field name | description | details   |
|---|---|---|
| Name | name of the lab | |
| Type | type of Fablab. | *Fab Lab, Service, Makerspace, DIYBio Lab, Ateneu de Fabricació, City Space, Library, FabCafè, Incubator/Accelerator. Foundation* | 
| Cluster | which cluster the lab belongs to (Barcelona or Milan) | *BCN, MI* |
| Address | The house number where the lab is located | |
| CAP | the zipcode of the address | | 
| Lat | the latitude expressed in decimal degree | | 
| Long | the longitude expressed in decimal degree | |
| Source | where the information was collected from | *fablabs.io, hackerspaces.org, makerspaces.make.co, ajuntament.barcelona.cat, sphere.diybio.org, manually added, ajuntament de Barcelona policy document* | 
| URL | the web address of the lab | | 
| Notes | some information notes.  |the only note is if the lab has been closed (*Closed*), otherwise nothing appears. | 

separation of active labs from closed ones

In [11]:
closed_labs = labs[labs.Notes.str.contains("losed")==True]

In [12]:
valid_labs = labs[~labs.index.isin(closed_labs.index)]

In [14]:
print("The total of the valid labs is %s" % valid_labs.shape[0])

The total of the valid labs is 62


show the first 10 in the dataset

In [15]:
valid_labs.head(10)

,Name,Type,City,Cluster,Address,CAP,Lat,Long,Source,URL,Notes
0,Fab Lab Barcelona,Fab Lab,Barcelona,BCN,"Carrer de Pujades, 102",8005.0,41.396925,2.194335,Fablabs.io,https://fablabbcn.org/,NaN
1,MADE Makerspace Barcelona,Fab Lab,Barcelona,BCN,Carrer Noguera Pallaresa 59-61,8014.0,41.373238,2.141084,"Fablabs.io, Hackerspaces.org, Makerspaces.make.co",NaN,NaN
2,Soko Tech,Fab Lab,Barcelona,BCN,"Carrer de Vallès i Ribot, 36",8027.0,41.424535,2.188883,Fablabs.io,https://soko.tech/,NaN
3,LSCongres,Fab Lab,Barcelona,BCN,"Carrer del Cardenal Tedeschini, 50",8027.0,41.426002,2.182026,Fablabs.io,https://fablabs.io/labs/lscongres,NaN
4,Green Fab Lab,Fab Lab,Cerdanyola del Vallès,BCN,"Ctra. BV-1415 (Horta-Cerdanyola), km 7",8290.0,41.450394,2.133441,Fablabs.io,NaN,NaN
5,Barcelona Mobility LAB,Fab Lab,Barcelona,BCN,Passatge del Torrent de l'Estadella 56,8030.0,41.432152,2.199793,Fablabs.io,NaN,NaN
6,Fab Lab Sant Cugat,Fab Lab,Sant Cugat del Vallès,BCN,"Avinguda de la Torre Blanca, 57",8712.0,41.467569,2.091704,Fablabs.io,NaN,NaN
8,The FabLab: Make in Milano,Fab Lab,Milan,MI,"Via Arcivescovo Calabiana, 6",20139.0,45.444046,9.207913,Fablabs.io,NaN,NaN
9,Vectorealism FabLab,Service,Milan,MI,"Via Carlo Boncompagni, 57",20139.0,45.438305,9.231811,Fablabs.io,https://fablabs.io/labs/vectorealismfablab,NaN
10,DamASpace,Fab Lab,Milan,MI,"Corso S. Gottardo, 19",20136.0,45.452457,9.179818,Fablabs.io,https://fablabs.io/labs/DamASpaceFablab,NaN


save the dataset on the filesystem

In [21]:
valid_labs.to_csv('data' + os.sep + 'valid_labs.csv')

## Calculation of the isochrones for each lab
the calculation is based on OpenRouteService using 15 minutes walking distance as the maximum time.

convert the Lat and Long in float to manage the geospatial information

In [22]:
valid_labs = valid_labs.astype({"Lat": float, "Long": float})

creation of the request for calculation of the isochrones to OpenRouteService according to the [API](https://openrouteservice.org/dev/#/api-docs/v2/isochrones) syntax

900 seconds = 15 minutes (city)


In [27]:
body = {"locations":[],"range":[900],"attributes":["total_pop","area","reachfactor"],"range_type":"time","interval":900,"area_units":"m","location_type":"start"}
locations = {}

In [28]:
# check if the latitude and longitude for each lab is valid
for index, row in valid_labs.iterrows():
    if np.isnan(row["Lat"]) == False and np.isnan(row["Long"]) == False:
      locations[row["Name"]] = [row["Long"], row["Lat"]]

In [33]:
print("We have %s where the latitude and longitude is present" % str(len(locations)))

We have 62 where the latitude and longitude is present


organize the header for the API request

In [35]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': api_key_ors,
    'Content-Type': 'application/json; charset=utf-8'
}

start with the requests.<Br/>
Attention:<Br/>
it is necessary to limit the number of requests per second

In [ ]:
body["locations"] = []
data = []
for k,i in enumerate(locations):
    print(i, locations[i])
    body["locations"].append(locations[i])
    # 15 minutes city, at walking distance
    call = requests.post('https://api.openrouteservice.org/v2/isochrones/foot-walking', json=body, headers=headers)
    body["locations"] = []
    print(call.status_code, call.reason)
    lab_dict = call.json()
    lab_dict["name"] = i
    data.append(lab_dict)
    print("---")
    time.sleep(30)

In [15]:
# Check the data, first lab
data[0]

{'type': 'FeatureCollection',
 'bbox': [2.181814, 41.386679, 2.206919, 41.408053],
 'features': [{'type': 'Feature',
   'properties': {'group_index': 0,
    'value': 900.0,
    'center': [2.1943253081862886, 41.396932544923246],
    'area': 4414692.78,
    'reachfactor': 0.8994,
    'total_pop': 85512.0},
   'geometry': {'coordinates': [[[2.181814, 41.394814],
      [2.181818, 41.394634],
      [2.181969, 41.39116],
      [2.182512, 41.388045],
      [2.182594, 41.387901],
      [2.182729, 41.387688],
      [2.182745, 41.387677],
      [2.186288, 41.387211],
      [2.189545, 41.387251],
      [2.192458, 41.386679],
      [2.194172, 41.38668],
      [2.199413, 41.386923],
      [2.20173, 41.387252],
      [2.201769, 41.38728],
      [2.20611, 41.391197],
      [2.206369, 41.391261],
      [2.206478, 41.3914],
      [2.20658, 41.39185],
      [2.206919, 41.395455],
      [2.206873, 41.397327],
      [2.206578, 41.402674],
      [2.205493, 41.406096],
      [2.205303, 41.406236],
      [2

store the data in the "data" folder as json file

In [16]:
with open("data" + os.sep + "isochrones.json", 'w') as outfile:
    json.dump(data, outfile)